In [36]:
import sys
sys.path.append("/Users/bubble/Desktop/Project/Infrasound Sensor/Layout/Code/acousticsensor")

import gdsfactory as gf
import Tools.geo_trans as gt


cell = gf.Component()

In [41]:
# structure for each side
L = [250, 500, 750, 1000]
g = [10, 20, 30]
w = 2
t = 0.5
beamNum = []
k = (len(L) * len(g))
mesh = [None] * k
for i in range(k):
    mesh[i] = gf.Component()
side = gf.Component()
count = 0
origin = (15000 / 13, 0)
for i in range(4):
    for j in range(3):
        # single grid
        row = column =  L[i] // (g[j]+w)
        beamNum.append(row+1)
        height = L[i] / row - w
        sg = gf.components.rectangle(size=(g[j], height), layer=(8, 0))
        for m in range(column):
            for n in range(row):
                sf_ref = mesh[count] << sg
                sf_ref.move((origin[0]+m*(g[j]+w), origin[1]+n*(height+w)))
        # frame
        fra_width = column * (g[j] + w) + w
        frame = gf.components.rectangle(size=(fra_width, L[i]), layer=(9, 0))
        frame_ref = mesh[count] << frame
        frame_ref.movex(origin[0]-w)
        # crystal
        mid = origin[0] - w + fra_width / 2
        cry_size = g[j] if j != 0 else 22
        cry_height = height+0.1 if j != 0 else 2*height+w+0.1
        crystal = gf.components.rectangle(size=(cry_size, cry_height), layer=(10, 0))
        crystal_ref = mesh[count] << crystal
        crystal_ref.move((mid-cry_size/2, L[i]-cry_height-w+0.1))
        # holes on crystal
        hole_size = 0.8
        hole_int = 0.9
        hole_dis = hole_int - hole_size
        hole_n = int(cry_size // hole_int) 
        hole_n = hole_n - 2 if hole_n % 2 == 0 else hole_n - 1
        print(hole_n)
        hole = gf.components.circle(radius=hole_size/2, layer=(11, 0))
        a = hole_n * hole_size + (hole_n-1) * hole_dis
        a = (cry_size - a) / 2 + hole_size / 2
        for m in range(hole_n):
            for n in range(hole_n):
                hole_ref = mesh[count] << hole
                hole_ref.move((mid-cry_size/2+a, L[i]-cry_height-w+0.1+a))
                hole_ref.move((m*hole_int, n*hole_int))
        # add mesh[count] to side and adjust position
        mesh_ref = side << mesh[count]
        mesh_ref.movex(count * 15000 / 13)        
        count += 1
# add some beams
for i in range(8):
    beam = gf.components.rectangle(size=(w, L[i//2]), layer=(9, 0))
    beam_ref = side << beam
    beam_ref.movex((i+1.5) * 15000 / 13+L[i//3]//2)


# side.show()
print(beamNum)

22
20
32
22
20
32
22
20
32
22
20
32
[21, 12, 8, 42, 23, 16, 63, 35, 24, 84, 46, 32]


In [38]:
# backside etching 15mm * 15mm
backside = gf.components.rectangle(size=(15000, 15000), layer=(2, 0))
# cell << backside
# cell.show()

In [39]:
# structure for each die

double_side = gf.Component()
# structure for double sides
side_ref = double_side << side
side_ref.dmirror_y(15000/2).dmirror_x(15000/2)
double_side << side

# structure for 4 sides
fside = gf.Component()
dside_ref = fside << double_side
dside_ref.drotate(angle=90, center=(15000/2, 15000/2))
fside << double_side
# fside.show()

Unnamed_269: ports [], vinsts=[] info=Info() kcl=KCLayout(name='DEFAULT', layout=<klayout.dbcore.Layout object at 0x10e367f50>, layer_enclosures=LayerEnclosureModel(root={'a4b5fcfc': LayerEnclosure(layer_sections={}, main_layer=8/0, yaml_tag='!Enclosure'), '4fb2b992': LayerEnclosure(layer_sections={}, main_layer=9/0, yaml_tag='!Enclosure'), 'c1ad96cb': LayerEnclosure(layer_sections={}, main_layer=10/0, yaml_tag='!Enclosure'), '5c226817': LayerEnclosure(layer_sections={}, main_layer=SLAB150 (2/0), yaml_tag='!Enclosure')}), cross_sections={'a4b5fcfc_10500': SymmetricalCrossSection(width=10500, enclosure=LayerEnclosure(layer_sections={}, main_layer=8/0, yaml_tag='!Enclosure'), name='a4b5fcfc_10500'), 'a4b5fcfc_10000': SymmetricalCrossSection(width=10000, enclosure=LayerEnclosure(layer_sections={}, main_layer=8/0, yaml_tag='!Enclosure'), name='a4b5fcfc_10000'), '4fb2b992_250000': SymmetricalCrossSection(width=250000, enclosure=LayerEnclosure(layer_sections={}, main_layer=9/0, yaml_tag='!En

In [40]:
# boolean operation
# do backside etching - frame
outside = gf.boolean(A = backside, B = fside,  operation="not", layer1=(2, 0), layer2=(9, 0), layer=(1, 0))
cell << outside

#  do holes etching - crystal
holes = gf.boolean(A = fside, B = fside,  operation="not", layer1=(8, 0), layer2=(10, 0), layer=(1, 0))
cell << holes
# add holes to cell
pc = gf.boolean(A = fside, B = fside,  operation="not", layer1=(11, 0), layer2=(30, 0), layer=(1, 0))
#  add backside etching to cell
cell << pc
cell << backside
cell.show()
# cell.write_gds("mesh.gds")
# cell.plot()